In [2]:
from langchain_core.language_models import BaseChatModel
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

model = ChatOllama(model="qwen3:4b", base_url="http://localhost:11434")
model.invoke([HumanMessage(content="Hi! I'm Bob")])

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


In [3]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}


In [4]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state



================================== Ai Message ==================================

<think>
Okay, the user introduced themselves as Bob. I need to respond in a friendly and engaging way. Let me start by acknowledging their name. Maybe add an emoji to keep it light.

I should ask how they're doing to start a conversation. Also, invite them to share more about themselves. Maybe ask about their interests or what they're up to. Keep the tone open and welcoming. Let me make sure the response is concise but inviting. Avoid any technical terms or complex questions. Keep it simple and conversational. Alright, that should work.
</think>

Hi there, Bob! 😊 How's your day going? What's on your mind? I'd love to hear more about you—what do you do, or what's been on your mind lately?


In [5]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user just asked, "What's my name?" Let me think about how to respond.

First, the user is Bob, as established in the previous message. But in this case, the user is asking for their own name. Wait, maybe there's a confusion here. The user might be referring to their real name, but in the context of the conversation, they introduced themselves as Bob. However, the user is now asking what their name is. That's a bit confusing. 

I need to clarify. The user might be confused about their own name in the context of the conversation. Or maybe they are testing if I remember that they said "Hi! I'm Bob." So, in the previous message, the user said "Hi! I'm Bob." Then the assistant responded with a greeting and asked about their day and what they're thinking. Now the user is asking "What's my name?" which is a bit of a loop. 

So, the correct response would be to point out that the user introduced

In [6]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user is asking, "What's my name?" I need to figure out how to respond. First, I should check if there's any information in the conversation history that might reveal their name. But looking at the history, the user hasn't provided any personal details. They just asked for their name, which is a bit confusing because I don't have access to personal information.

Wait, maybe they're testing if I can retrieve their name from somewhere. But according to the guidelines, I shouldn't use personal information. So I should inform them that I don't have access to their name. Also, they might be confused because they think their name is known to me, but in reality, I don't have that data.

I should respond politely, explaining that I don't have access to their name. Maybe offer to help them find it if they have any clues. But I need to make sure I don't make up a name or assume anything. Also, chec

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [9]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [10]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user introduced themselves as Jim. I need to respond in a pirate-like manner. Let me think of a greeting that's both fun and in character.

"Arrr, matey! What's yer name, then? I be Captain Scurvybeard, and I be a bit of a troublemaker, but I be friendly to the bone! What brings ye to me ship, Jim?" 

That sounds good. It's playful, uses pirate terms, and invites conversation. I should keep the tone consistent and make sure to use phrases like "matey," "arrr," and "ye" to maintain the pirate persona. Also, adding a bit of humor with "a bit of a troublemaker, but I be friendly to the bone!" adds character. I need to make sure the response is engaging and encourages the user to continue the interaction.
</think>

Aye, matey! What's yer name, then? I be Captain Scurvybeard, and I be a bit of a troublemaker, but I be friendly to the bone! What brings ye to me ship, Jim? Do ye seek treasure, 

In [11]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user asked, "What is my name?" after introducing himself as Jim. Let me think about how to respond in a pirate style.

First, the user is Jim, so I need to confirm that. The previous response was in a pirate tone, so I should maintain that. The user might be testing if I remember their name or just want a playful response. Since they introduced themselves as Jim, I should acknowledge that. Maybe add a bit of humor or a pirate-related twist. Also, the user might be looking for a confirmation or a playful interaction. I should make sure to keep the tone consistent with the pirate character, using words like "matey," "arrr," and maybe a bit of a riddle or a playful question. Let me check if there's any need to add more depth or if it's just a straightforward answer. The user might be expecting a simple "Jim" but with a pirate flair. I should respond with something like "Aye, matey! Yer name

In [12]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [13]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [14]:
config = {"configurable": {"thread_id": "abc458"}}
query = "Hi! I'm Bob."
language = "Chinese"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user introduced himself as Bob. I need to respond in Chinese. Let me start by greeting him. "Hi Bob!" is a good start. Then, I should ask how he's doing to keep the conversation going. Maybe something like "你好，Bob！最近怎么样？" That sounds friendly and open. I should make sure my Chinese is correct. Also, I want to invite him to ask questions or share thoughts. Maybe add, "有什么问题或者想聊的话题吗？" to encourage interaction. Let me check the flow again. Yes, that should work. I'll send that response.
</think>

你好，Bob！最近怎么样？有什么问题或者想聊的话题吗？


In [15]:
from langchain_core.messages import SystemMessage, trim_messages, AIMessage

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

d:\anaconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


OSError: Can't load tokenizer for 'gpt2'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'gpt2' is the correct path to a directory containing all relevant files for a GPT2TokenizerFast tokenizer.